In [ ]:
%matplotlib inline
#%% NumPyの読み込み
import numpy as np
#   NumPyのLinalgモジュールの読み込み
import numpy.linalg as lin
#   CVXPYの読み込み
import cvxpy as cvx
#   Pandasの読み込み
import pandas as pd
#   MatplotlibのPyplotモジュールの読み込み
import matplotlib.pyplot as plt
#   日本語フォントの設定
from matplotlib.font_manager import FontProperties
import sys
if sys.platform.startswith('win'):
    FontPath = 'C:\\Windows\\Fonts\\meiryo.ttc'
elif sys.platform.startswith('darwin'):
    FontPath = '/System/Library/Fonts/ヒラギノ角ゴシック W4.ttc'
elif sys.platform.startswith('linux'):
    FontPath = '/usr/share/fonts/truetype/takao-gothic/TakaoPGothic.ttf'
else:
    print('このPythonコードが対応していないOSを使用しています．')
    sys.exit()
jpfont = FontProperties(fname=FontPath)
import math
import japanize_matplotlib
from scipy.stats import norm

In [ ]:
cd /Users/shintaro/Downloads/

In [ ]:
marketcap = pd.read_csv('efficient_frontier(ver1.8).csv', encoding = 'Shift_Jis')
del marketcap['キヤノン']
data_date = marketcap['銘柄名称']
del marketcap['銘柄名称']

In [ ]:
marketcap

In [ ]:
Data2 = (marketcap.diff()/marketcap.shift(1))[1:] * 100 

In [ ]:
Data2

In [ ]:
Mu = Data2.mean().values
Stdev = Data2.std().values

In [ ]:
Asset_Weight = [0.029, 
0.256, 
0.036, 
0.014, 
0.496, 
0.007, 
0.034, 
0.018, 
0.006, 
0.006, 
0.006, 
0.01, 
0.03, 
0.017, 
0.035]
sum(Asset_Weight)

In [ ]:
Asset_Weight

In [ ]:
column = Data2.columns.tolist()
column.pop(1)
column.pop(0)

In [ ]:
for i in range(15):
    Data2[column[i]] = (Data2[column[i]] * Asset_Weight[i])

In [ ]:
for i in range(15):
    if i == 0:
        Data2['Portfolio'] = Data2[column[i]]
    
    else:
        Data2['Portfolio'] = Data2['Portfolio'] + Data2[column[i]]

In [ ]:
Data2.to_csv('Portfolio_Return_AC.csv', encoding='Shift_JIS')

In [ ]:
df = pd.DataFrame(data=Data2['日経平均'])
df['TOPIX'] = Data2['TOPIX']
df['Portfolio'] = Data2['Portfolio']

In [ ]:
data_date.pop(0)

In [ ]:
df.set_axis(data_date, axis=0, inplace=True)

In [ ]:
df.to_csv('Portfolio_Return_ac.csv', encoding='Shift_JIS')

In [ ]:
for i in range (3):
    for j in range(len(df)):
        if j == 0:
            df.iloc[j, i] = df.iloc[j, i]
            
        else:
            df.iloc[j, i] = df.iloc[j, i] + df.iloc[j - 1, i]

# 累積収益率のグラフ

In [ ]:
df.plot(alpha = 0.5)
plt.xlabel('Date')
plt.ylabel('累積収益率')
plt.title('パフォーマンス比較')
#plt.savefig('パフォーマンス比較_累積(ver1.4).png', dpi = 1500)

# リターン

In [ ]:
Return = df.mean().tolist()
Return

In [ ]:
Portfolio_Return = []

for i in range(15):
    Portfolio_Return.append(Mu[i + 2] * Asset_Weight[i])
    
sum(Portfolio_Return)

# リスク

In [ ]:
Risk = df.std().tolist()
Risk

In [ ]:
R = (marketcap.diff()/marketcap.shift(1))[1:] 
del R['日経平均']
del R['TOPIX']

Assets_Cov = R.cov()
Asset_Weight = np.array(Asset_Weight)

In [ ]:
RIsk_mc = np.sqrt(np.dot(Asset_Weight.T, np.dot(Assets_Cov, Asset_Weight)))

# シャープレシオ

In [ ]:
sharp_ratio = []

for i in range(3):
    sharp_ratio.append((Return[i] - 0.017448518)/Risk[i])

In [ ]:
sharp_ratio

In [ ]:
df

In [ ]:
data1 = df['日経平均'].tolist()
data2 = df['TOPIX'].tolist()
data3 = df['Portfolio'].tolist()

In [ ]:
data1_tmp = []
data2_tmp = []
data3_tmp = []

for i in range(len(df)):
    if data1[i] < Return[0]:
        
        data1_tmp.append(Return[0] - data1[i])
        
for i in range(len(df)):
    if data2[i] < Return[1]:
        
        data2_tmp.append(Return[1] - data2[i])
        
for i in range(len(df)):
    if data3[i] < Return[2]:
        
        data3_tmp.append(Return[2] - data3[i])

In [ ]:
Under_Stdev1 = np.mean(data1_tmp)
Under_Stdev2 = np.mean(data2_tmp)
Under_Stdev3 = np.mean(data3_tmp)

In [ ]:
Under_Stdev = []
Under_Stdev.append(Under_Stdev1)
Under_Stdev.append(Under_Stdev2)
Under_Stdev.append(Under_Stdev3)

In [ ]:
Under_Stdev

In [ ]:
sortino_ratio = []

for i in range(3):
    sortino_ratio.append((Return[i] - 0.017448518)/Under_Stdev[i])

# ソルティノレシオ

In [ ]:
sortino_ratio

# デルタ法によるVaRの算出

In [ ]:
import datetime
import numpy as np
import pandas as pd
from scipy.stats import norm
from pandas_datareader import data as pdr
import yfinance as yf
import requests_cache
# override pandas_datareader
yf.pdr_override()
# casche for downloaded data
expire_after = datetime.timedelta(days=3)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
def download_data_adjclose(stock, start, end): # 株価データの取得
    data =  pdr.get_data_yahoo(stock, start, end)['Adj Close']
    return data
def calc_daily_pct_change(data):  # 日次リターンの計算
    pct_change = data.pct_change()
    return pct_change
class ValueAtRisk(object):
    def __init__(self, s, c, n= 1, stock='GOOGL', start='2015-01-01', end='2020-01-01'):
        self.s = s
        self.c = c
        
        start = pd.to_datetime(start) 
        end = pd.to_datetime(end)
        data = download_data_adjclose(stock, start, end)
        pct_change = calc_daily_pct_change(data)
        self.mu = np.mean(pct_change) * n
        self.sigma = np.std(pct_change) * np.sqrt(n)
    def calc(self):
        alpha = norm.ppf(1-self.c)
        var = self.s * (self.mu - self.sigma*alpha)
        return var
if __name__ == '__main__':
    s = 1000000
    c = 0.99
    value_at_risk = ValueAtRisk(s, c)
    # 35662.65269757571
    print(value_at_risk.calc())

# モンテカルロ・シミュレーション法によるVaRの算出 𝑆𝑡=𝑆0exp((𝜇−𝜎22)𝑡+𝜎𝐵𝑡)
・上がデルタ法・下がモンテカルロ・シミュレーション法

In [ ]:
import datetime
import numpy as np
import pandas as pd
from scipy.stats import norm
from pandas_datareader import data as pdr
import yfinance as yf
import requests_cache
# override pandas_datareader
yf.pdr_override()
# casche for downloaded data
expire_after = datetime.timedelta(days=3)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
def download_data_adjclose(stock, start, end):
    data =  pdr.get_data_yahoo(stock, start, end)['Close']
    return data
def calc_daily_pct_change(data):
    pct_change = data.pct_change()
    return pct_change
class ValueAtRisk(object):
    def __init__(self, s, c, n= 1, stock='^N225', start='2016-07-01', end='2021-08-01'):
        self.s = s
        self.c = c
        
        start = pd.to_datetime(start) 
        end = pd.to_datetime(end)
        data = download_data_adjclose(stock, start, end)
        pct_change = calc_daily_pct_change(data)
        self.mu = np.mean(pct_change) * n
        self.sigma = np.std(pct_change) * np.sqrt(n)
    def calc(self):
        alpha = norm.ppf(1-self.c)
        var = self.s * (self.mu - self.sigma*alpha)
        return var
    def monte_carlo(self, iterations=100000):
        np.random.seed(seed=0)
        
        # generate stock prices 
        rands = np.random.normal(0, 1, [1, iterations])
        stock_prices = self.s * np.exp((self.mu-0.5*self.sigma**2) + self.sigma*rands)
        
        # sort an array of stock_prices and get (1-self.c)*100 percentile
        # e.x. VaR with 99% confidence level, there is left 1% probability less than VaR.
        stock_prices.sort()
        percentile = np.percentile(stock_prices, (1-self.c)*100)
        
        return self.s - percentile
if __name__ == '__main__':
    s = 5000000
    c = 0.99
    value_at_risk = ValueAtRisk(s, c)
    # 35662.65269757571
    print(value_at_risk.calc())
    # 33607.08215287316
    print(value_at_risk.monte_carlo())
    print(((value_at_risk.monte_carlo()/5000000)*100))

In [ ]:
# ライブラリの読み込み
import yfinance as yf

# 証券コードを指定する
# 日本株の場合は、証券コードの最後に'.T'を付ける
# トヨタ7203の場合は、'7203.T'となる
stock_code = '151'

# 株情報の取得開始日を指定する
start = '2016-07-01'

# 株情報の取得終了日を指定する
end = '2021-08-01'

# 期間を指定して株情報を取得する
toyota1 = yf.download(tickers=stock_code, start=start, end=end)
print(toyota1)

# 直近、1日間の30分間隔の株情報を取得する
# periodでは、1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,maxを設定できる
# intervalでは、1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3moを設定できる
toyota2 = yf.download(tickers=stock_code, period='10y', interval='1mo')
print(toyota2)


In [ ]:
toyota2.to_csv('/Users/shintaro/Downloads/7912.csv', encoding='Shift_JIS')

In [ ]:
import pandas as pd

In [ ]:
Data = pd.read_csv('/Users/shintaro/Downloads/efficient_frontier(ver1.7).csv', encoding='Shift_JIS')
del Data['銘柄名称']

In [ ]:
Data

In [ ]:
Data2 = (Data.diff()/Data.shift(1))[1:] * 100

In [ ]:
Data2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
Data3 = Data2['京セラ'].tolist()

In [ ]:
Assets = Data2.columns.tolist()

for i in range(len(Assets)):
    Data3 = Data2[Assets[i]].tolist()
    plt.hist(Data3)
    plt.title(Assets[i])
    plt.show()

In [ ]:
Data4 = Data['大日本印刷'].tolist()

In [ ]:
Data4

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import math
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from numpy.random import *
import pandas as pd

In [ ]:
font_path = '/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc'
font_prop = FontProperties(fname=font_path)
matplotlib.rcParams['font.family'] = font_prop.get_name()

In [ ]:
fp = FontProperties(fname=font_path, size=14)
df = pd.read_csv('/Users/shintaro/Downloads/efficient_frontier(ver1.7).csv', encoding='shift_jis')

In [ ]:
df

In [ ]:
df['diff'] = df['日経平均'].diff(periods=1)

In [ ]:
df

In [ ]:
df['diff'] = df['diff'].fillna(0)
plt.figure()
df['diff'].plot.hist(bins=100)
plt.xlabel(u'前日との差',fontproperties=fp)
plt.ylabel(u'頻度',fontproperties=fp)

### ヒストリカル法

In [ ]:
df['diff'].quantile(1.0-0.99)

### デルタ法

In [ ]:
df['diff'].std() * 2.33

In [ ]:
df['diff'].mean()

In [ ]:
marketcap.pct_change() * 100

In [ ]:
marketcap

In [ ]:
Data2

In [ ]:
pct_change_all = marketcap.pct_change()
pct_change_all.drop(index=[0], inplace=True)
pct_change = np.array(pct_change_all['日経平均'].tolist())

In [ ]:
def calc_VaR_delta_and_montecarlo(pct_change, n, c, s, iteration, L1, L2, price):
    mu = pct_change.mean() * n
    sigma = pct_change.std() * np.sqrt(n)
    
    alpha = norm.ppf(1 - c)
    var = s * (mu - sigma * alpha)
    
    print((var/price)*100)
    L1.append((var/price)*100)
    
    np.random.seed(seed=0)
    rands = np.random.normal(0, 1, [1, iteration])
    stock_prices = s * np.exp((mu - 0.5*sigma**2) + sigma*rands)
    percentile = np.percentile(stock_prices, (1 - c) * 100)
    
    print(((s - percentile)/price)*100)
    L2.append(((s - percentile)/price)*100)

In [ ]:
def calc_VaR_delta_and_montecarlo(pct_change, n, c, s, iteration, L1, L2, price):
    
    mu = pct_change.mean() * n
    sigma = pct_change.std() * np.sqrt(n)
    
    alpha = norm.ppf(1 - c)
    var = s * (mu - sigma * alpha)
    
    print((var/price)*100)
    L1.append((var/price)*100)
    
    np.random.seed(seed=0)
    rands = np.random.normal(0, 1, [1, iteration])
    stock_prices = s * np.exp((mu - 0.5*sigma**2) + sigma*rands)
    stock_prices.sort()
    percentile = np.percentile(stock_prices, (1 - c) * 100)
    
    print(((s - percentile)/price)*100)
    L2.append((s - percentile))

In [ ]:
column = pct_change_all.columns.tolist()

In [ ]:
VaR_N225_and_TOPIX_delta =[]
VaR_N225_and_TOPIX_MC =[]

for i in range(2):
    pct_change = np.array(pct_change_all[column[i]])
    calc_VaR_delta_and_montecarlo(pct_change, 5, 0.95, 5000000, 100000, VaR_N225_and_TOPIX_delta, VaR_N225_and_TOPIX_MC, 5000000)

In [ ]:
column

In [ ]:
VaR_N225_and_TOPIX_MC

In [ ]:
sum((Asset_Weight*5000000))#.tolist())

In [ ]:
VaR_Portfolio_delta = []
VaR_Portfolio_MC = []

for i in range(15):
    pct_change = np.array(pct_change_all[column[i + 2]])
    Asset_price = (Asset_Weight[i]*5000000)
    calc_VaR_delta_and_montecarlo(pct_change, 1, 0.95,Asset_price, 1000000, VaR_Portfolio_delta, VaR_Portfolio_MC, Asset_price)

In [ ]:
VaR_Portfolio_MC

In [ ]:
(sum(VaR_Portfolio_MC)/15)

In [ ]:
pct_change_all.std()

In [ ]:
Asset_Weight*5000000

In [ ]:
n=1
c=0.95
s=5000000
price=5000000
L1=[]
L2=[]
iteration=10000

mu = Return[2] * n/100
sigma = RIsk_mc * np.sqrt(n)
    
alpha = norm.ppf(1 - c)
var = s * (mu - sigma * alpha)
    
print((var/price)*100)
L1.append((var/price)*100)
    
np.random.seed(seed=0)
rands = np.random.normal(0, 1, [1, iteration])
stock_prices = s * np.exp((mu - 0.5*sigma**2) + sigma*rands)
stock_prices.sort()
percentile = np.percentile(stock_prices, (1 - c) * 100)
    
print(((s - percentile)/price)*100)
L2.append((s - percentile))

In [ ]:
n=1
c=0.95
s=5000000
price=5000000
L1=[]
L2=[]
iteration=10000

mu = Return[0] * n/100
sigma = Risk[0]/100 * np.sqrt(n)
    
alpha = norm.ppf(1 - c)
var = s * (mu - sigma * alpha)
    
print((var/price)*100)
L1.append((var/price)*100)
    
np.random.seed(seed=0)
rands = np.random.normal(0, 1, [1, iteration])
stock_prices = s * np.exp((mu - 0.5*sigma**2) + sigma*rands)
stock_prices.sort()
percentile = np.percentile(stock_prices, (1 - c) * 100)
    
print(((s - percentile)/price)*100)
L2.append((s - percentile))